### Instalar las dependeciencias necesarias para el proyecto

In [ ]:
%pip install langchain
%pip install openai
%pip install python-dotenv
%pip install "unstructured[local-inference]"
%pip install chromadb
%pip install tiktoken

### Cargar las variables de entorno

Recuerda renombrar el archivo `.env.example` a `.env.local` y rellenar los campos con tus credenciales.

In [ ]:
%load_ext dotenv
%dotenv

### Cargar libro

Libro descargado desde el proyecto Elejandría

[Crimen y Castigo de Dostoyevski](https://www.elejandria.com/libro/crimen-y-castigo/dostoyevski-fiodor/146)

In [ ]:
from langchain.document_loaders import UnstructuredPDFLoader

In [ ]:
loader = UnstructuredPDFLoader("crimen_y_castigo.pdf")
data = loader.load()

In [ ]:
print (f'Existen {len(data)} documento(s) en tu archivo')
print (f'Con un total de {len(data[0].page_content)} caracteres')

### Dividir el documento en bloques mas pequeños

In [ ]:
from langchain.text_splitter import RecursiveCharacterTextSplitter

In [ ]:
text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=0)
texts = text_splitter.split_documents(data)

In [ ]:
print (f'Despues de dividir el documento, ahora existen {len(texts)} textos')

### Embeddings de los bloques

Crear los embeddings a partir de los los textos generados en el paso anterior.
Esto lo hago para que la búsqueda semántica sea mas eficiente y ademas evitamos la perdida de contexto por parte de GPT.

Base de datos usada: [Chroma](https://www.trychroma.com/)

In [ ]:
from langchain.embeddings.openai import OpenAIEmbeddings
from langchain.vectorstores import Chroma

In [ ]:
embeddings = OpenAIEmbeddings()
db = Chroma.from_documents(texts, embeddings)

In [ ]:
testing_query = "¿Cuál es el crimen que comete el protagonista, Rodion Raskólnikov, y por qué lo comete?"
testing_docs = db.similarity_search(testing_query, include_metadata=True)

In [ ]:
print (f'Documentos encontrados que se ajustan a la pregunta: {len(docs)} documento(s)')
print(f'Contenido del primer documento: {testing_docs[0].page_content}')

### Pregunta y respuesta
Usando la base de datos creada en el paso anterior, realizamos la pregunta a GPT

In [ ]:
from langchain.llms import OpenAI
from langchain.chains.question_answering import load_qa_chain

In [ ]:
llm = OpenAI(temperature=0)
chain = load_qa_chain(llm, chain_type="stuff")

In [ ]:
query = "¿Cuál es el crimen que comete el protagonista, Rodion Raskólnikov, y por qué lo comete?"
docs = db.similarity_search(query, include_metadata=True)

In [ ]:
chain.run(input_documents=docs, question=query)

La respuesta es algo similar a esto:
> Rodion Raskólnikov comete el asesinato y el robo de la vieja prestamista y su hermana Lisbeth. Lo comete por la miseria y el deseo de abrirse paso en la vida con los tres mil rublos que esperaba encontrar en casa de la víctima.

Es importante destacar que **NO** estamos realizando un búsqueda "clásica" dentro del libro, sino que estamos usando como contexto para GPT la base de datos de Chroma que contiene multiples fragmentos del libro almacenados en forma de vectores.

Ahora podemos hablar con un libro, y que el libro nos conteste.

¡Se acabaron los deberes escolares!